In [55]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import numpy as np
import urllib.request 
from mip import Model, xsum, maximize, BINARY, LinExpr

In [58]:
def CTFPFunction(graphUrl, skillsUrl, secondarySkillsUrl, numTeams, teamUsersProportion, graphSep = "\n", numUsersSep = " ", skillsSep = "\n", numSkillsSep = " "):

  #1 - CRIANDO MATRIZ DE ADJACÊNCIA DAS RELAÇÕES ENTRE OS USUÁRIOS
  graph = urllib.request.urlopen(graphUrl).read().decode('utf-8').split(graphSep)
  del graph[-1] #vazio

  numUsers = int(graph[0].split(numUsersSep)[0])
  relationships = np.zeros(numUsers*numUsers, dtype=np.int64).reshape(numUsers,numUsers)

  for line in graph[1:]:
    l = [x for x in line.split(" ") if x]
    l[0] = int(l[0])
    l[1] = int(l[1])
    l[2] = round(float(l[2]))
    if int(l[0]) < numUsers and int(l[1]) < numUsers:
      relationships[l[0],l[1]] = relationships[l[1],l[0]] = int(l[2])

  print(f'Número usuários: {numUsers}')
  print(f'Matriz de relacionamentos {relationships}')
  print(f'Densidade do grafo: {np.count_nonzero(relationships)/(numUsers*(numUsers-1))}')

  unique, counts = np.unique(relationships, return_counts=True)

  print(f'Número de arestas positivas e negativas: {dict(zip(unique, counts/2))}')

  print('\n')


  #2 - DEFININDO CONJUNTO DE SKILLS DE CADA USUÁRIO
  skillsData = urllib.request.urlopen(skillsUrl).read().decode('utf-8').split(skillsSep)
  numSkills = int(skillsData[0].split(numSkillsSep)[0])

  print(f'Número de skills: {numSkills}')
  lstSkills = skillsData[0].split(numSkillsSep)[1:]
  mapSkills = {int(lstSkills[i].split(":")[0]):lstSkills[i].split(":")[1] for i in range(len(lstSkills))}
  print(f'Lista de skills: {mapSkills}')
  print('\n')


  #3 - CONTANDO O NÚMERO DE USUÁRIOS PARA CADA SKILL
  usersSkillsMatrix = np.zeros(numUsers*numSkills, dtype=np.int64).reshape(numUsers,numSkills)

  for index,item in enumerate(skillsData[1:]):
    usersSkillsMatrix[index] = np.asarray(item.split(" "))

  numUsersBySkill = usersSkillsMatrix.sum(axis=0)


  #4 - FUNÇÃO QUE RETORNA O CONJUNTO DE SKILLS DE UM USUÁRIO
  def userSkillSet(user):
    return np.asarray([id for id,s in enumerate(skillsData[user+1].split(" ")) if s and int(s) == 1])

  
  #5 - DEFININDO DEMANDA DE USUÁRIOS POR SKILL PARA CADA TIME
  teamDemand = np.full((numTeams, numSkills), 1, dtype=np.int64)

  for index,item in enumerate(numUsersBySkill):
    for num in range(numTeams):
      teamDemand[num][index] = round(item/teamUsersProportion)

  print(f'Demanda de cada time: {teamDemand}')
  print('\n')

  #6 - CONJUNTO DE SKILLS QUE CADA TIME PRECISA
  def demandSkillSet(team):
    return np.asarray([id for id,s in enumerate(teamDemand[team]) if s > 0])


  #7 - IMPLEMENTAÇÃO DO MODELO
  model = Model("CTFP")

  #7.1 - Variáveis
  x = [[[model.add_var(var_type=BINARY) for s in range(numSkills)] for u in range(numUsers)] for t in range(numTeams)]

  #7.2 - Função objetivo
  model.objective = maximize(xsum( xsum( xsum( x[t][u][s] for s in userSkillSet(u) ) for u in range(numUsers) ) for t in range(numTeams)))

  #7.3 - Restrições

  #restrição (63)
  for t in range(numTeams):
    for u in range(numUsers):
      for v in range(numUsers):
        if relationships[u,v] == -1:
          model += xsum(x[t][u][s] for s in userSkillSet(u)) + xsum(x[t][v][s] for s in userSkillSet(v)) <= 1

  #restrição (64)
  for t1 in range(numTeams):
    for t2 in np.arange(t1+1, numTeams, 1):
      for u in range(numUsers):
        for v in range(numUsers):
          if relationships[u,v] == 1:
            model += xsum(x[t1][u][s] for s in userSkillSet(u)) + xsum(x[t2][v][s] for s in userSkillSet(v)) <= 1

  #restrição (65)
  for u in range(numUsers):
    model += xsum( xsum( x[t][u][s] for s in userSkillSet(u)) for t in range(numTeams)) <= 1

  #restrição (66)
  for t in range(numTeams):
    for s in demandSkillSet(t):
      model += xsum( x[t][u][s] for u in range(numUsers) if s in userSkillSet(u)) >= teamDemand[t,s]

  model.optimize()

  print('\n')

  result = model.objective_value

  print(f'Total de indivíduos alocados: {result}\n')

  if result == None:
    return

  #8 - PRINTANDO RESULTADOS

  # Mapeando skills secundárias
  secondarySkillsData = urllib.request.urlopen(secondarySkillsUrl).read().decode('utf-8').split(skillsSep)
  numSecondarySkills = int(secondarySkillsData[0].split(numSkillsSep)[0])

  lstSecondarySkills = secondarySkillsData[0].split(numSkillsSep)[1:]
  mapSecondarySkills = {int(lstSecondarySkills[i].split(":")[0]):lstSecondarySkills[i].split(":")[1] for i in range(len(lstSecondarySkills))}

  # Definindo conjunto de skill secundárias de cada usuário
  def userSecondarySkillSet(user):
    return np.asarray([id for id,s in enumerate(secondarySkillsData[user+1].split(" ")) if s and int(s) == 1])

  for teamIdx,team in enumerate(x):
    
    usersCount = 0
    skillsUsersCount = [0] * numSkills
    secondarySkillsUserCount = [0] * numSecondarySkills

    for userIdx,user in enumerate(team):
    
      for skillIdx,skill in enumerate(user):
        
        skillsUsersCount[skillIdx] += skill.x # Contando número de usuários alocados na skill
        usersCount += skill.x #Contando número de usuários alocados em uma equipe
    
        for s in userSecondarySkillSet(userIdx):
          secondarySkillsUserCount[s] += skill.x # Contando número de usuários alocados na skill secundária
      
    print(f'Resultados da equipe {teamIdx}')

    skillsUsersCountStr = ' | '.join(mapSkills[ind]+': '+str(count) for ind, count in enumerate(skillsUsersCount))
    secondarySkillsUsersCountStr = ' | '.join(mapSecondarySkills[ind]+': '+str(count) for ind, count in enumerate(secondarySkillsUserCount))
    print(f'Número de indivíduos na equipe: {usersCount}')
    print(f'Número de indíviduos em cada skill: {skillsUsersCountStr}')
    print(f'Número de indíviduos em cada skill secundária: {secondarySkillsUsersCountStr}')

    print('\n')

  # 9 - ESCREVENDO ARQUIVO CONTENDO A EQUIPE E SKILL EM QUE CADA USUÁRIO ESTÁ ALOCADO
  fName = f'{graphUrl.split("/")[-1].split(".")[0]}_{skillsUrl.split("/")[-2]}_{str(numTeams)}-teams.csv'

  output = open(fName, "w")
  output.write('id_vertex; team; skill')
  output.close()

  outputLines = [{'team': '-', 'skill': '-'}] * numUsers

  for teamIdx,team in enumerate(x):

    for userIdx,user in enumerate(team):
    
      skillUsed = -1

      for skillIdx,skill in enumerate(user):
        if skill.x == 1: # o usuário estará na equipe com somente 1 skill
          skillUsed = skillIdx #armazena a skill que o usuário está usando na equipe

      # caso o usuário esteja alocado na equipe, anotará seu id, a equipe e a skill em que está alocado
      if skillUsed != -1:
        outputLines[userIdx] = {'team': str(teamIdx), 'skill': mapSkills[skillUsed] }

  output = open(fName, "a")
  for id,item in enumerate(outputLines):
    output.write('\n')
    output.write(f'{str(id)}; {item["team"]}; {item["skill"]}')
  output.close()

# 2010

## Estado como skill

In [ ]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2010-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2010.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2010.txt',
             2,
             5,
             numUsersSep="\t")

Número usuários: 545
Matriz de relacionamentos [[0 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'SP', 2: 'RJ', 3: 'BA', 4: 'PE', 5: 'RR', 6: 'MS', 7: 'PR', 8: 'SC', 9: 'RS', 10: 'MT', 11: 'ES', 12: 'DF', 13: 'CE', 14: 'TO', 15: 'GO', 16: 'AP', 17: 'PA', 18: 'AM', 19: 'RO', 20: 'AC', 21: 'AL', 22: 'MA', 23: 'RN', 24: 'PB', 25: 'SE', 26: 'PI'}


Demanda de cada time: [[11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]
 [11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]]




Total de indivíduos alocados: 331.0

Resultados da equipe 0
Número de indivíduos na equipe: 157.0
Número de indíviduos em cada skill: MG: 14.0 | SP: 23.0 | RJ: 19.0 | BA: 11.0 | PE: 7.0 | RR: 2.0 | MS: 4.0 | PR: 10.0 | SC: 4.0 | RS: 8.0 | MT: 2.0 | ES: 6.0 | DF: 3.0 | CE: 7.0 | TO: 2.0 | GO: 3.0 | AP: 3.0 | PA: 4.0 | AM: 2.0 | RO

In [ ]:
# TRÊS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2010-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2010.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2010.txt',
             3,
             4,
             numUsersSep="\t")

Número usuários: 545
Matriz de relacionamentos [[0 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]]
Densidade do grafo: 0.22331354560172692
Número de arestas positivas e negativas: {-1: 389.0, 0: 115408.5, 1: 32715.0}


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'SP', 2: 'RJ', 3: 'BA', 4: 'PE', 5: 'RR', 6: 'MS', 7: 'PR', 8: 'SC', 9: 'RS', 10: 'MT', 11: 'ES', 12: 'DF', 13: 'CE', 14: 'TO', 15: 'GO', 16: 'AP', 17: 'PA', 18: 'AM', 19: 'RO', 20: 'AC', 21: 'AL', 22: 'MA', 23: 'RN', 24: 'PB', 25: 'SE', 26: 'PI'}


Demanda de cada time: [[13 18 13 11  6  2  2  8  4  8  2  2  2  6  2  4  2  4  2  2  2  2  5  2
   3  2  3]
 [13 18 13 11  6  2  2  8  4  8  2  2  2  6  2  4  2  4  2  2  2  2  5  2
   3  2  3]
 [13 18 13 11  6  2  2  8  4  8  2  2  2  6  2  4  2  4  2  2  2  2  5  2
   3  2  3]]




## Partido como skill

In [ ]:
# DUAS EQUIPES
# sem resultados até o tempo limite
CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2010-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2010.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2010.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 545
Matriz de relacionamentos [[0 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]]
Densidade do grafo: 0.22331354560172692
Número de arestas positivas e negativas: {-1: 389.0, 0: 115408.5, 1: 32715.0}


Número de skills: 19
Lista de skills: {0: 'DEM', 1: 'PRB', 2: 'PT', 3: 'PTB', 4: 'PSB', 5: 'PMDB', 6: 'PP', 7: 'PCDOB', 8: 'PSDB', 9: 'PR', 10: 'PPS', 11: 'PDT', 12: 'PSOL', 13: 'PSC', 14: 'PHS', 15: 'PV', 16: 'PTC', 17: 'PMN', 18: 'PTDOB'}


Demanda de cada time: [[20  3 28  8  9 32 15  4 20 15  5  8  1  6  1  5  1  1  0]
 [20  3 28  8  9 32 15  4 20 15  5  8  1  6  1  5  1  1  0]]




# 2011

## Estado como skill

In [60]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2011-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2011.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2011.txt',
             2,
             5,
             numUsersSep="\t")

Número usuários: 553
Matriz de relacionamentos [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 0 0]]
Densidade do grafo: 0.031632465864716824
Número de arestas positivas e negativas: {-1: 246.0, 0: 148076.5, 1: 4582.0}


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'BA', 2: 'PE', 3: 'DF', 4: 'CE', 5: 'PI', 6: 'AM', 7: 'RJ', 8: 'SP', 9: 'PR', 10: 'SC', 11: 'RS', 12: 'ES', 13: 'GO', 14: 'MT', 15: 'AP', 16: 'PA', 17: 'AC', 18: 'TO', 19: 'RR', 20: 'MA', 21: 'RN', 22: 'PB', 23: 'AL', 24: 'SE', 25: 'MS', 26: 'RO'}


Demanda de cada time: [[11  8  5  2  4  2  2 10 15  6  4  6  2  4  2  2  4  2  2  2  4  2  2  2
   2  2  2]
 [11  8  5  2  4  2  2 10 15  6  4  6  2  4  2  2  4  2  2  2  4  2  2  2
   2  2  2]]




Total de indivíduos alocados: 535.0

Resultados da equipe 0
Número de indivíduos na equipe: 133.0
Número de indíviduos em cada skill: MG: 11.0 | BA: 9.0 | PE: 5.0 | DF: 2.0 | CE: 15.0 | PI: 2.0 | AM: 2.0 | RJ: 10.

In [ ]:
# TRÊS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2011-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2011.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2011.txt',
             3,
             3.8,
             numUsersSep="\t")

Número usuários: 553
Matriz de relacionamentos [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'BA', 2: 'PE', 3: 'DF', 4: 'CE', 5: 'PI', 6: 'AM', 7: 'RJ', 8: 'SP', 9: 'PR', 10: 'SC', 11: 'RS', 12: 'ES', 13: 'GO', 14: 'MT', 15: 'AP', 16: 'PA', 17: 'AC', 18: 'TO', 19: 'RR', 20: 'MA', 21: 'RN', 22: 'PB', 23: 'AL', 24: 'SE', 25: 'MS', 26: 'RO'}


Demanda de cada time: [[14 11  7  3  6  3  2 14 19  8  6  8  3  6  3  2  5  2  2  2  6  2  3  2
   2  2  2]
 [14 11  7  3  6  3  2 14 19  8  6  8  3  6  3  2  5  2  2  2  6  2  3  2
   2  2  2]
 [14 11  7  3  6  3  2 14 19  8  6  8  3  6  3  2  5  2  2  2  6  2  3  2
   2  2  2]]




## Partido como skill

In [ ]:
# DUAS EQUIPES
#sem resultados até o tempo limite
CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2011-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2011.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2011.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 553
Matriz de relacionamentos [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 0 0]]
Densidade do grafo: 0.031632465864716824
Número de arestas positivas e negativas: {-1: 246.0, 0: 148076.5, 1: 4582.0}


Número de skills: 23
Lista de skills: {0: 'DEM', 1: 'PT', 2: 'PTB', 3: 'PR', 4: 'PMDB', 5: 'PP', 6: 'PCDOB', 7: 'PPS', 8: 'PSOL', 9: 'PDT', 10: 'PRB', 11: 'PSDB', 12: 'PSB', 13: 'PSD', 14: 'PSC', 15: 'PV', 16: 'PMN', 17: 'PTDOB', 18: 'PHS', 19: 'PRTB', 20: 'PSL', 21: 'PTC', 22: 'PRP'}


Demanda de cada time: [[13 30  7 12 30 15  5  4  1 10  4 19 11  8  6  4  1  1  1  1  0  0  1]
 [13 30  7 12 30 15  5  4  1 10  4 19 11  8  6  4  1  1  1  1  0  0  1]]




# 2012

## Estado como skill

In [ ]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2012-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2012.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2012.txt',
             2,
             3.2,
             numUsersSep="\t")

Número usuários: 555
Matriz de relacionamentos [[0 1 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
Densidade do grafo: 0.3040491755293199
Número de arestas positivas e negativas: {-1: 398.0, 0: 107269.5, 1: 46345.0}


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'BA', 2: 'PE', 3: 'DF', 4: 'CE', 5: 'PI', 6: 'AM', 7: 'RJ', 8: 'SP', 9: 'PR', 10: 'SC', 11: 'RS', 12: 'ES', 13: 'MT', 14: 'GO', 15: 'AP', 16: 'PA', 17: 'AC', 18: 'TO', 19: 'RR', 20: 'MA', 21: 'RN', 22: 'PB', 23: 'AL', 24: 'SE', 25: 'MS', 26: 'RO'}


Demanda de cada time: [[17 13  8  3  8  3  2 17 23 10  5 11  3  3  6  2  6  2  2  2  7  3  4  3
   2  2  2]
 [17 13  8  3  8  3  2 17 23 10  5 11  3  3  6  2  6  2  2  2  7  3  4  3
   2  2  2]]




## Partido como skill

In [ ]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2012-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2012.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2012.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 555
Matriz de relacionamentos [[0 1 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
Densidade do grafo: 0.3040491755293199
Número de arestas positivas e negativas: {-1: 398.0, 0: 107269.5, 1: 46345.0}


Número de skills: 24
Lista de skills: {0: 'DEM', 1: 'PT', 2: 'PTB', 3: 'PR', 4: 'PMDB', 5: 'PP', 6: 'PPS', 7: 'PSB', 8: 'PSOL', 9: 'PDT', 10: 'PSDB', 11: 'PSD', 12: 'PSC', 13: 'PV', 14: 'PCDOB', 15: 'PRB', 16: 'PMN', 17: 'PTDOB', 18: 'PHS', 19: 'PRTB', 20: 'PSL', 21: 'PTC', 22: 'PEN', 23: 'PRP'}


Demanda de cada time: [[10 31  7 12 29 13  4 12  1  9 19 17  6  3  4  3  1  1  0  0  0  0  0  1]
 [10 31  7 12 29 13  4 12  1  9 19 17  6  3  4  3  1  1  0  0  0  0  0  1]]




Total de indivíduos alocados: None



# 2013

## Estado como skill

In [61]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2013-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2013.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2013.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 540
Matriz de relacionamentos [[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Densidade do grafo: 0.026187040472754757
Número de arestas positivas e negativas: {-1: 933.0, 0: 141989.0, 1: 2878.0}


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'PE', 2: 'DF', 3: 'CE', 4: 'PI', 5: 'AM', 6: 'RJ', 7: 'SP', 8: 'PR', 9: 'SC', 10: 'RS', 11: 'ES', 12: 'TO', 13: 'MT', 14: 'GO', 15: 'BA', 16: 'AP', 17: 'PA', 18: 'RO', 19: 'AC', 20: 'RR', 21: 'MA', 22: 'RN', 23: 'PB', 24: 'AL', 25: 'SE', 26: 'MS'}


Demanda de cada time: [[19  8  3  8  3  3 16 24 11  6 11  3  4  3  6 13  3  6  3  3  3  6  3  4
   3  3  3]
 [19  8  3  8  3  3 16 24 11  6 11  3  4  3  6 13  3  6  3  3  3  6  3  4
   3  3  3]]




Total de indivíduos alocados: 522.0

Resultados da equipe 0
Número de indivíduos na equipe: 325.0
Número de indíviduos em cada skill: MG: 33.0 | PE: 17.0 | DF: 6.0 | CE: 13.0 | PI: 6.0 | AM: 5.0 | RJ: 30.0 | SP: 4

In [ ]:
# TRÊS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2013-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2013.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2013.txt',
             3,
             3.7,
             numUsersSep="\t")

Número usuários: 540
Matriz de relacionamentos [[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'PE', 2: 'DF', 3: 'CE', 4: 'PI', 5: 'AM', 6: 'RJ', 7: 'SP', 8: 'PR', 9: 'SC', 10: 'RS', 11: 'ES', 12: 'TO', 13: 'MT', 14: 'GO', 15: 'BA', 16: 'AP', 17: 'PA', 18: 'RO', 19: 'AC', 20: 'RR', 21: 'MA', 22: 'RN', 23: 'PB', 24: 'AL', 25: 'SE', 26: 'MS'}


Demanda de cada time: [[15  7  2  6  3  2 13 19  9  5  9  3  3  2  5 11  2  5  2  2  2  5  2  3
   3  2  2]
 [15  7  2  6  3  2 13 19  9  5  9  3  3  2  5 11  2  5  2  2  2  5  2  3
   3  2  2]
 [15  7  2  6  3  2 13 19  9  5  9  3  3  2  5 11  2  5  2  2  2  5  2  3
   3  2  2]]




## Partido como skill

In [62]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2013-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2013.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2013.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 540
Matriz de relacionamentos [[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Densidade do grafo: 0.026187040472754757
Número de arestas positivas e negativas: {-1: 933.0, 0: 141989.0, 1: 2878.0}


Número de skills: 25
Lista de skills: {0: 'DEM', 1: 'PSC', 2: 'PSDB', 3: 'PMDB', 4: 'PP', 5: 'SOLIDARIED', 6: 'PT', 7: 'PTB', 8: 'PRB', 9: 'PPS', 10: 'PSB', 11: 'PSOL', 12: 'PDT', 13: 'PR', 14: 'PEN', 15: 'PSD', 16: 'PV', 17: 'PCDOB', 18: 'SDD', 19: 'S.PART.', 20: 'PMN', 21: 'PTDOB', 22: 'PHS', 23: 'PRP', 24: 'PROS'}


Demanda de cada time: [[10  6 16 29 13  2 31  6  3  4  9  1  8 12  1 16  3  4  1  0  1  1  0  1
   0]
 [10  6 16 29 13  2 31  6  3  4  9  1  8 12  1 16  3  4  1  0  1  1  0  1
   0]]




Total de indivíduos alocados: 521.0

Resultados da equipe 0
Número de indivíduos na equipe: 223.0
Número de indíviduos em cada skill: DEM: 10.0 | PSC: 6.0 | PSDB: 16.0 | PMDB: 39.0 | PP: 20.0 | SOLIDARIED:

# 2014

## Estado como skill

In [ ]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2014-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2014.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2014.txt',
             2,
             3.5,
             numUsersSep="\t")

# 2015

## Estado como skill

In [63]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2015-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2015.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2015.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 552
Matriz de relacionamentos [[0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 1]
 [1 0 1 ... 0 1 0]]
Densidade do grafo: 0.025217654330729374
Número de arestas positivas e negativas: {-1: 775.0, 0: 148517.0, 1: 3060.0}


Número de skills: 27
Lista de skills: {0: 'PE', 1: 'DF', 2: 'CE', 3: 'RR', 4: 'SP', 5: 'PI', 6: 'PA', 7: 'BA', 8: 'RJ', 9: 'PR', 10: 'RS', 11: 'MG', 12: 'GO', 13: 'MA', 14: 'SC', 15: 'AP', 16: 'PB', 17: 'AL', 18: 'AM', 19: 'ES', 20: 'RO', 21: 'MS', 22: 'TO', 23: 'SE', 24: 'MT', 25: 'RN', 26: 'AC'}


Demanda de cada time: [[ 9  3  8  3 26  5  6 14 18 10 11 18  6  7  6  3  4  3  3  3  3  3  3  3
   3  3  3]
 [ 9  3  8  3 26  5  6 14 18 10 11 18  6  7  6  3  4  3  3  3  3  3  3  3
   3  3  3]]




Total de indivíduos alocados: 538.0

Resultados da equipe 0
Número de indivíduos na equipe: 329.0
Número de indíviduos em cada skill: PE: 16.0 | DF: 5.0 | CE: 17.0 | RR: 4.0 | SP: 51.0 | PI: 7.0 | PA: 11.0 | BA: 2

## Partido como skill

In [64]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2015-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2015.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2015.txt',
             2,
             2.5,
             numUsersSep="\t")

Número usuários: 552
Matriz de relacionamentos [[0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 1]
 [1 0 1 ... 0 1 0]]
Densidade do grafo: 0.025217654330729374
Número de arestas positivas e negativas: {-1: 775.0, 0: 148517.0, 1: 3060.0}


Número de skills: 30
Lista de skills: {0: 'PSC', 1: 'PSDB', 2: 'PMDB', 3: 'PSB', 4: 'PRB', 5: 'PP', 6: 'PTN', 7: 'PT', 8: 'PTB', 9: 'PPS', 10: 'PDT', 11: 'PSOL', 12: 'DEM', 13: 'SOLIDARIED', 14: 'PSD', 15: 'PR', 16: 'PCDOB', 17: 'PROS', 18: 'PV', 19: 'PHS', 20: 'PTDOB', 21: 'PSDC', 22: 'PRTB', 23: 'PTC', 24: 'PSL', 25: 'PEN', 26: 'PRP', 27: 'PMN', 28: 'REDE', 29: 'S.PART.'}


Demanda de cada time: [[ 6 22 30 14  8 16  2 28 10  5  9  2  9  7 15 14  5  6  4  2  1  1  0  1
   0  1  1  1  0  0]
 [ 6 22 30 14  8 16  2 28 10  5  9  2  9  7 15 14  5  6  4  2  1  1  0  1
   0  1  1  1  0  0]]




Total de indivíduos alocados: 537.0

Resultados da equipe 0
Número de indivíduos na equipe: 260.0
Número de indíviduo

# 2016

## Estado como skill

In [ ]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2016-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2016.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2016.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 544
Matriz de relacionamentos [[0 0 0 ... 1 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]]


Número de skills: 27
Lista de skills: {0: 'PE', 1: 'DF', 2: 'CE', 3: 'RR', 4: 'SP', 5: 'PI', 6: 'PA', 7: 'BA', 8: 'RJ', 9: 'PR', 10: 'RS', 11: 'MG', 12: 'GO', 13: 'MA', 14: 'SC', 15: 'MT', 16: 'AP', 17: 'PB', 18: 'AL', 19: 'AM', 20: 'ES', 21: 'RO', 22: 'MS', 23: 'TO', 24: 'SE', 25: 'RN', 26: 'AC'}


Demanda de cada time: [[10  3  8  3 24  4  6 13 17 11 11 19  6  6  6  3  3  4  3  3  3  3  3  3
   3  3  3]
 [10  3  8  3 24  4  6 13 17 11 11 19  6  6  6  3  3  4  3  3  3  3  3  3
   3  3  3]]




Total de indivíduos alocados: 497.0

Resultados da equipe 0
Número de indivíduos na equipe: 311.0
Número de indíviduos em cada skill: PE: 15.0 | DF: 5.0 | CE: 12.0 | RR: 4.0 | SP: 45.0 | PI: 6.0 | PA: 10.0 | BA: 25.0 | RJ: 28.0 | PR: 15.0 | RS: 19.0 | MG: 33.0 | GO: 12.0 | MA: 12.0 | SC: 10.0 | MT: 6.0 | AP: 4.0 | PB: 8.0 | AL: 4

## Partido como skill

In [ ]:
# DUAS EQUIPES

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2016-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2016.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2016.txt',
             2,
             3,
             numUsersSep="\t")

Número usuários: 544
Matriz de relacionamentos [[0 0 0 ... 1 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]]
Densidade do grafo: 0.0416531253385332
Número de arestas positivas e negativas: {-1: 657.0, 0: 141816.0, 1: 5495.0}


Número de skills: 29
Lista de skills: {0: 'PTDOB', 1: 'PSDB', 2: 'PSB', 3: 'PP', 4: 'PSC', 5: 'PTN', 6: 'PMDB', 7: 'PT', 8: 'PTB', 9: 'PRB', 10: 'PPS', 11: 'PDT', 12: 'PSOL', 13: 'DEM', 14: 'SOLIDARIED', 15: 'PSD', 16: 'PR', 17: 'PCDOB', 18: 'PROS', 19: 'PV', 20: 'REDE', 21: 'PEN', 22: 'PSL', 23: 'PMB', 24: 'PHS', 25: 'PTC', 26: 'S.PART.', 27: 'PMN', 28: 'PRP'}


Demanda de cada time: [[ 1 18 11 16  5  4 24 20  7  7  3  7  2  9  6 11 14  4  2  2  2  1  1  2
   1  0  0  0  0]
 [ 1 18 11 16  5  4 24 20  7  7  3  7  2  9  6 11 14  4  2  2  2  1  1  2
   1  0  0  0  0]]


